# Comparing DuckDB, Polars, and Pandas

I am going to use duckdb, polars and pandas to do some analysis on some csv files and I am going to compare the performance.

In [1]:
# first we are going to import the python libraries

import pandas as pd
import polars as pl
import duckdb
import time

## Car Price Dataset

The first csv file that I am going to use for benchmarking purposes can be found here: https://www.kaggle.com/datasets/asinow/car-price-dataset

I am specifically going to try to find the second most expensive model for each brand for every year.

In [23]:
df = pd.read_csv('car_price_dataset.csv')
print(df.shape)

(10000, 10)


This dataset has 10k rows

In [8]:
def pandas_car_price_func():
    df = pd.read_csv('car_price_dataset.csv')
    out = df.sort_values('Price', ascending=False).groupby(['Brand','Year'], as_index=False).nth(2) [["Brand", "Model", "Year", "Price"]]
    out.to_csv('car_price_pandas.csv')

In [9]:
%%timeit -n10 -r5
pandas_car_price_func()

19.5 ms ± 1.19 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)


In [10]:
def polars_car_price_func():
    df = pl.read_csv('car_price_dataset.csv')
    df = df.select('Brand', 'Model', 'Year', 'Price', pl.col('Price').rank('ordinal', descending=True).over('Brand','Year').alias('price_rank'))
    df = df.filter(pl.col("price_rank") == 2).select('Brand', 'Model', 'Year', 'Price')
    df.write_csv('car_price_polars.csv')

In [11]:
%%timeit -n10 -r5
polars_car_price_func()

5.73 ms ± 831 μs per loop (mean ± std. dev. of 5 runs, 10 loops each)


In [12]:
def duckdb_car_price_func():
    duckdb.read_csv("car_price_dataset.csv")
    duckdb.sql('''
    with temp_data as (
        SELECT
            Brand,
            Model,
            Year,
            Price,
            row_number() over (partition by Brand, Year order by Price desc) as rn
        FROM 'car_price_dataset.csv'
    )
    SELECT
        Brand,
        Model,
        Year,
        Price
    FROM temp_data
    WHERE rn = 2
    ''').to_csv('car_price_duckdb.csv')
    

In [13]:
%%timeit -n10 -r5
duckdb_car_price_func()

142 ms ± 4.35 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)


DuckDB gives the worst results and polars gives the best for this dataset

## Spotify Dataset

Let's use a larger dataset for comparison this time. This time we are going to use the spotify dataset that can be found here: https://www.kaggle.com/datasets/anandshaw2001/top-spotify-songs-in-73-countries

We are specifically going to look for the 3rd most popular song per country

In [24]:
df = pd.read_csv('Top_spotify_songs.csv')
print(df.shape)

(1728382, 25)


This dataset has 1.7m rows

In [14]:
def pandas_spotify_func():
    df = pd.read_csv('Top_spotify_songs.csv')
    out = df.sort_values('popularity', ascending=False).groupby('country', as_index=False).nth(3) [["name", "artists", "country", "popularity"]]
    out.to_csv('spotify_pandas.csv')

In [15]:
%%timeit -n10 -r1
pandas_spotify_func()

7.37 s ± 0 ns per loop (mean ± std. dev. of 1 run, 10 loops each)


In [16]:
def polars_spotify_func():
    df = pl.read_csv('Top_spotify_songs.csv')
    df = df.select('name', 'artists', 'country', 'popularity', pl.col('popularity').rank('ordinal', descending=True).over('country').alias('popularity_rank'))
    df = df.filter(pl.col("popularity_rank") == 3).select('name', 'artists', 'country', 'popularity')
    df.write_csv('spotify_polars.csv')

In [17]:
%%timeit -n10 -r1
polars_spotify_func()

784 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 10 loops each)


In [18]:
def duckdb_spotify_func():
    duckdb.read_csv("Top_spotify_songs.csv")
    duckdb.sql('''
    with temp_data as (
        SELECT
            name,
            artists,
            country,
            popularity,
            row_number() over (partition by country order by popularity desc) as rn
        FROM 'Top_spotify_songs.csv'
    )
    SELECT
        name,
        artists,
        country,
        popularity
    FROM temp_data
    WHERE rn = 3
    ''').to_csv('spotify_duckdb.csv')

In [19]:
%%timeit -n10 -r1
duckdb_spotify_func()

1.64 s ± 0 ns per loop (mean ± std. dev. of 1 run, 10 loops each)


Using Polars results in the fastest execution time but this time using DuckDB is better than using Pandas

## Amazon Dataset

This time we are going to use the book review file from the following link: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews?select=Books_rating.csv

In [25]:
df = pd.read_csv('Books_rating.csv')
print(df.shape)

(3000000, 10)


In [26]:
df.columns

Index(['Id', 'Title', 'Price', 'User_id', 'profileName', 'review/helpfulness',
       'review/score', 'review/time', 'review/summary', 'review/text'],
      dtype='object')

In [31]:
def pandas_amazon_func():
    df = pd.read_csv('Books_rating.csv')
    out = df.sort_values('review/score', ascending=False).groupby('User_id', as_index=False).nth(1) [["Title", "review/score", "User_id"]]
    out.to_csv('book_data_pandas.csv')

In [37]:
%%timeit -n5 -r1
pandas_amazon_func()

45.2 s ± 0 ns per loop (mean ± std. dev. of 1 run, 5 loops each)


In [38]:
def polars_amazon_func():
    df = pl.read_csv('Books_rating.csv')
    df = df.select('Title', 'review/score', 'User_id', pl.col('review/score').rank('ordinal', descending=True).over('User_id').alias('score_rank'))
    df = df.filter(pl.col("score_rank") == 1).select('Title', 'review/score', 'User_id')
    df.write_csv('book_data_polars.csv')

In [39]:
%%timeit -n5 -r1
polars_amazon_func()

5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 5 loops each)


In [42]:
def duckdb_amazon_func():
    duckdb.read_csv("Books_rating.csv")
    duckdb.sql('''
    with temp_data as (
        SELECT
            Title,
            'review/score',
            User_id,
            row_number() over (partition by User_id order by 'review/score' desc) as rn
        FROM 'Books_rating.csv'
    )
    SELECT
        Title,
        'review/score',
        User_id
    FROM temp_data
    WHERE rn = 1
    ''').to_csv('book_data_duckdb.csv')

In [43]:
%%timeit -n5 -r1
duckdb_amazon_func()

6.64 s ± 0 ns per loop (mean ± std. dev. of 1 run, 5 loops each)
